In [9]:
# code for automatically reloading changes from modules (used for testing)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
file_list = [
    ('H1-1', '2022_06_09'),
    ('H1-2', '2022_06_09'),
    ('H1-1', '2023_01_20'),
    ('H2-1', '2023_03_10'),
    ('H1-1', '2023_07_17'),
    ('H1-2', '2023_08_21'),
    ('H1-1', '2024_04_10'),
    ('H2-1', '2024_05_20'),
    ('H2-2', '2024_12_06'),
]

## Emulator paramater csv

In [11]:
# First row
import csv

fields=[
    'Date', 
    'Machine', 
    'p1', 
    'p1_emission_ratio', 
    'p2', 
    'p2_emission_ratio', 
    'dephasing_error', 
    'p_crosstalk_meas', 
    'p_crosstalk_init', 
    'p_meas_0', 
    'p_meas_1'
]
with open(r'Emulator parameters.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

In [13]:
import csv
import pathlib
from qtm_spec.combined_analysis import emulator_parameters

for machine, date in file_list:
    field_name = [
        'p1', 
        'p1_emission_ratio', 
        'p2', 
        'p2_emission_ratio', 
        'dephasing_error', 
        'p_crosstalk_meas', 
        'p_crosstalk_init', 
        'p_meas_0', 
        'p_meas_1'
    ]

    data_dir = pathlib.Path.cwd().parent.joinpath('data')
    emulator_dict = emulator_parameters(data_dir, machine, date)
    fields = (
        [date, machine] +
        [emulator_dict[key] if key in emulator_dict else None for key in field_name]
    )
    with open('Emulator parameters.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)

In [23]:
from pandas import read_csv
read_csv('Emulator parameters.csv')

## Spec sheet parameters csv

In [34]:
# First row
import csv

fields=[
    'Date', 
    'Machine', 
    '1Q error', 
    '1Q error (legacy)',
    '1Q leakage', 
    '2Q error', 
    '2Q error (legacy)',
    '2Q leakage', 
    'Transport 1Q error', 
    'MCM error', 
    'MCR error', 
    'SPAM error', 
]
with open(r'Spec sheet parameters.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

In [35]:
import csv
import pathlib
from qtm_spec.combined_analysis import extract_parameters
from uncertainties import ufloat

for machine, date in file_list:
    field_name = [
        '1Q error', 
        '1Q error (legacy)'
        '1Q leakage', 
        '2Q error', 
        '2Q error (legacy)',
        '2Q leakage', 
        'Transport 1Q error', 
        'MCM error', 
        'MCR error', 
        'SPAM error', 
    ]

    data_dir = pathlib.Path.cwd().parent.joinpath('data')
    renamed = {
        'SQ_RB': '1Q error',
        'TQ_RB': '2Q error',
        'SQ_RB_legacy': '1Q error (legacy)',
        'TQ_RB_legacy': '2Q error (legacy)',
        'SQ_RB_leakage': '1Q leakage',
        'TQ_RB_leakage': '2Q leakage',
        'Memory_RB_legacy': 'Transport 1Q error',
        'Measurement_crosstalk': 'MCM error',
        'Reset_crosstalk': 'MCR error',
        'SPAM': 'SPAM error'
    }
    df_raw = extract_parameters(data_dir, machine, date)

    spec_dict = {}
    for old_name, new_name in renamed.items():
        if old_name in df_raw:
            spec_dict[new_name] = '{:.1uES}'.format(ufloat(df_raw[old_name][0], df_raw[old_name][1]))
        else:
            spec_dict[new_name] = None

    fields = (
        [date, machine] +
        [spec_dict[key] if key in spec_dict else None for key in field_name]
    )
    with open(r'Spec sheet parameters.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)

In [33]:
df_raw

{'SQ_RB_legacy': [7.266659492677352e-05, 2.1958959282752932e-05],
 'SQ_RB_leakage': [1.1590159040286707e-05, 3.693362975398795e-06],
 'SQ_RB': [7.846167444691687e-05, 2.2036472614704303e-05],
 'TQ_RB_legacy': [0.0012922230079112085, 0.00010886861903086631],
 'TQ_RB_leakage': [0.00042533169145408944, 6.492599056889814e-05],
 'TQ_RB': [0.0013985559307747308, 0.00011007196611943227],
 'Memory_RB_legacy': [0.0005297075295169762, 3.956482010769591e-05],
 'Measurement_crosstalk': [6.264077197665351e-06, 8.625283175994536e-07],
 'SPAM0': [0.0009000000000001229, 0.000299864969611344],
 'SPAM1': [0.0017625000000000002, 0.00041945125983241485],
 'SPAM': [0.001331250000000006, 9.114863537359807e-05]}

In [36]:
from pandas import read_csv
read_csv('Spec sheet parameters.csv')

,Date,Machine,1Q error,1Q error (legacy)1Q leakage,2Q error,2Q error (legacy),2Q leakage,Transport 1Q error,MCM error,MCR error,SPAM error
0,2022_06_09,H1-1,NaN,NaN,NaN,2.40(7)E-03,NaN,3.4(2)E-04,1.2(2)E-05,NaN,2.8(1)E-03
1,2022_06_09,H1-2,NaN,NaN,NaN,2.50(9)E-03,NaN,3.4(2)E-04,5.2(4)E-05,NaN,3.2(2)E-03
2,2023_01_20,H1-1,4.5(8)E-05,NaN,2.15(7)E-03,2.05(7)E-03,4.2(3)E-04,1.4(3)E-04,7(5)E-06,NaN,3.1(4)E-03
3,2023_03_10,H2-1,2.7(3)E-05,NaN,1.93(5)E-03,1.83(5)E-03,3.9(2)E-04,2.2(3)E-04,4.4(7)E-06,NaN,1.6(1)E-03
4,2023_07_17,H1-1,3.2(5)E-05,NaN,1.47(7)E-03,1.38(7)E-03,3.8(3)E-04,1.8(2)E-04,8.7(8)E-06,NaN,2.7(1)E-03
5,2023_08_21,H1-2,6(1)E-05,NaN,3.0(1)E-03,3.0(1)E-03,3.1(3)E-04,4.3(3)E-04,3.4(2)E-05,NaN,3.2(1)E-03
6,2024_04_10,H1-1,2.4(3)E-05,NaN,9.3(3)E-04,8.8(3)E-04,2.1(1)E-04,2.1(2)E-04,1.5(1)E-05,NaN,2.5(1)E-03
7,2024_05_20,H2-1,3.4(4)E-05,NaN,1.36(9)E-03,1.28(9)E-03,3.3(4)E-04,5.0(3)E-04,7.4(8)E-06,NaN,1.5(1)E-03
8,2024_12_06,H2-2,8(2)E-05,NaN,1.4(1)E-03,1.3(1)E-03,4.3(7)E-04,5.3(4)E-04,6.3(8)E-06,NaN,1.33(9)E-03
